# Creating Customers Gold Table


In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 7556e6d7-9177-4df6-b7b0-be6577c73b92, 3, Finished, Available, Finished)

In [2]:
%%sql
create table if not exists ecom_proj_LH.gold_customer
(
    Customer_ID string,
    Customer_Name string,
    Segment string,
    City string,
    State string,
    Country string,
    Region string,
    Created_TS timestamp,
    Modified_TS timestamp
)
using DELTA

StatementMeta(, 7556e6d7-9177-4df6-b7b0-be6577c73b92, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
Max_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from ecom_proj_LH.gold_customer").first()[0]

StatementMeta(, 7556e6d7-9177-4df6-b7b0-be6577c73b92, 5, Finished, Available, Finished)

In [4]:
Max_Date 

StatementMeta(, 7556e6d7-9177-4df6-b7b0-be6577c73b92, 6, Finished, Available, Finished)

'1900-01-01'

In [5]:
df_bronze=spark.read.table("ecom_proj_LH.bronze_sales")
df_final = df_bronze.selectExpr(
    "Customer_ID" ,
    "Customer_Name" ,
    "Segment" ,
    "City" ,
    "State" ,
    "Country" ,
    "Region" ,
).where(col("Modified_TS")>Max_Date).drop_duplicates()

StatementMeta(, 7556e6d7-9177-4df6-b7b0-be6577c73b92, 7, Finished, Available, Finished)

In [7]:
df_final.createOrReplaceTempView("ViewCustomer")

StatementMeta(, 7556e6d7-9177-4df6-b7b0-be6577c73b92, 9, Finished, Available, Finished)

In [ ]:
# Merging the data into gold customer table

In [10]:
%%sql
Merge into ecom_proj_LH.gold_customer as gc
using ViewCustomer as vc
on gc.Customer_ID = vc.Customer_ID
when matched then 
update set
gc.Customer_Name	=	vc.Customer_Name	,
gc.Segment	=	vc.Segment	,
gc.City	=	vc.City	,
gc.State	=	vc.State	,
gc.Country	=	vc.Country	,
gc.Region	=	vc.Region	,
gc.Modified_TS	=	current_timestamp()	

when not matched then insert 
(
gc.Customer_ID	,
gc.Customer_Name	,
gc.Segment	,
gc.City	,
gc.State	,
gc.Country	,
gc.Region	,
gc.Created_TS	,
gc.Modified_TS	
)
values 
(
vc.Customer_ID	,
vc.Customer_Name	,
vc.Segment	,
vc.City	,
vc.State	,
vc.Country	,
vc.Region	,
current_timestamp(),
current_timestamp
)

StatementMeta(, 7556e6d7-9177-4df6-b7b0-be6577c73b92, 12, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>